In [ ]:
LAB_USER_NAME = 'REPLACE_ME'

In [ ]:
import panel as pn
import sys
import os
import yaml
import numpy as np
from bokeh.palettes import Viridis
import asyncio

from matipo import Sequence, SEQUENCE_DIR, GLOBALS_DIR, DATA_DIR, Unit
from matipo.util.decimation import decimate
from matipo.util.fft import fft_reconstruction
from matipo.util.etl import deinterlace
from matipo.util import ilt

from matipo.experiment.base_experiment import BaseExperiment, auto_inputs, PlotInterface
from matipo.experiment.models import PLOT_COLORS, SITickFormatter, SIHoverFormatter
from matipo.experiment.plots import SignalPlot, SpectrumPlot, ImagePlot, Image1DPlot, ComplexDataLinePlot

pn.extension()

WORKSPACE = os.path.join(LAB_USER_NAME, 'SRCPMG Map')
LAB_DIR = os.path.join(DATA_DIR, WORKSPACE)
os.makedirs(LAB_DIR, exist_ok=True)
print('Data will be saved to', LAB_DIR)

In [ ]:
from loglogmap_plot import LogLogMapPlot
from ilt_SRCPMG import SRCPMG_T1T2_spectrum

In [ ]:
class T1_T2_SRCPMG(BaseExperiment):
    
    def setup(self):
        self.enable_partialplot = True
        self.title = "T1-T2 SRCPMG Experiment"
        self.workspace = WORKSPACE
        self.seq = Sequence('SRCPMG.py')
        self.plots = {
            'map': LogLogMapPlot(
                interpolate=10,
                figure_opts = dict(
                            title="T1-T2 Map",
                            x_axis_label="T2",
                            y_axis_label="T1",
                )),
            'recovery': ComplexDataLinePlot(
                figure_opts = dict(
                        title="T1 Saturation Recovery",
                        x_axis_label="Time (s)",
                        y_axis_label="Amplitude")),
            'decay': ComplexDataLinePlot(
                figure_opts = dict(
                            title="T2 decay",
                            x_axis_label="Time (s)",
                            y_axis_label="Amplitude"))
        }

        self.inputs = auto_inputs(self.seq, {
            'n_scans': 2,
            'n_rec': 10,
            't_rec_min': 2e-4*Unit('s'),
            't_rec_max': 2*Unit('s'),
            't_echo': 200*Unit('us'),
            'n_echo': 5000,
            'n_samples': 16,
            't_dw': 4*Unit('us')
        })        
        
    def update_par(self):     
        self.seq.setpar(
            t_end=0.1, # don't need to wait for T1 recovery in saturation recovery type experiments, should still wait >>T2* to avoid coherence pathways
            t_sat=5e-3*np.array([7, 5, 3, 2]), # pulse spacings should be larger than T2* and decreasing (recommended by literature)
            t_rec=np.logspace(np.log10(self.inputs['t_rec_max'].value), np.log10(self.inputs['t_rec_min'].value), self.inputs['n_rec'].value) # measure using recovery times in decreasing order (recommended by literature)
        )
        
    async def update_plots(self, final):
        await self.seq.fetch_data()
        
        # average the echos to get the integrated echo decay curve
        y = np.reshape(np.mean(np.reshape(self.seq.data,(-1, self.seq.par.n_samples)),axis=1),(-1, self.seq.par.n_echo))
        
        # correct phase assuming smallest inversion time has a phase of 180
        phase = np.angle(np.mean(y[0, :2]))
        y *= np.exp(1j*-phase)
        
        # make time axes
        t_T2 = np.linspace(0, self.seq.par.n_echo*self.seq.par.t_echo, self.seq.par.n_echo)
        t_T1 = self.seq.par.t_rec
        
        self.plots['recovery'].update(t_T1[:self.progress.value], np.mean(y[:self.progress.value, :2], axis=1))
        self.plots['decay'].update(t_T2, y[self.progress.value-1])
        self.log.debug(f'self.progress.value: {self.progress.value}')
        
        if final:
            self.log.info('Running Inverse Laplace Transform (~1 min)...')
            # Run Inverse Laplace Transform
            n_T1 = 50
            n_T2 = 50
            alpha = 1
            self.T1 = np.logspace(-3,1,n_T1)
            self.T2 = np.logspace(-3,1,n_T2)
            self.S = SRCPMG_T1T2_spectrum(self.T1, self.T2, t_T1, t_T2, y, alpha, tol=1e-5, progress=1000)

            self.plots['map'].update(self.S, self.T2[0], self.T2[-1], self.T1[0], self.T1[-1])

SRCPMGExp = T1_T2_SRCPMG(state_id='srcpmg')
SRCPMGExp()